In [85]:
import ujson

from itertools import islice
from tqdm import tqdm_notebook
from glob import glob
from collections import Counter

from gensim.models import KeyedVectors

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.metrics import r2_score

In [4]:
w2v = KeyedVectors.load_word2vec_format(
    '../data/vectors/GoogleNews-vectors-negative300.bin.gz',
    binary=True,
)

In [35]:
class Corpus:
    
    def __init__(self, pattern, skim=None):
        self.pattern = pattern
        self.skim = skim
        
    def lines(self):
        for path in glob(self.pattern):
            with open(path) as fh:
                for line in fh:
                    yield line.strip()

    def abstracts(self):
        lines = self.lines()
        if self.skim:
            lines = islice(lines, self.skim)
        for line in tqdm_notebook(lines, total=self.skim):
            raw = ujson.loads(line)
            yield Abstract.from_raw(raw)
            
    def xy(self):
        for abstract in self.abstracts():
            yield from abstract.xy()

In [40]:
class Abstract:
    
    @classmethod
    def from_raw(cls, raw):
        return cls([Sentence(s['token']) for s in raw['sentences']])
    
    def __init__(self, sentences):
        self.sentences = sentences
            
    def xy(self):
        for i, sent in enumerate(self.sentences):
            x = sent.token_vectors()
            y = i / (len(self.sentences)-1)
            yield x, y

In [41]:
class Sentence:
    
    def __init__(self, tokens):
        self.tokens = tokens
        
    def token_vectors(self):
        return [w2v[t] for t in self.tokens if t in w2v]

In [44]:
train = Corpus('../data/train.json/*.json', 10000)

In [45]:
train_x, train_y = zip(*train.xy())

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))

In [101]:
train_x = pad_sequences(train_x, 50, padding='post', dtype=float)

In [102]:
train_y = list(train_y)

In [103]:
model = Sequential()
model.add(LSTM(128, input_shape=train_x[0].shape))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [104]:
model.fit(train_x, train_y, batch_size=10)

Epoch 1/10
53739/53739 [==============================] - 195s 4ms/step - loss: 0.1241
Epoch 2/10
53739/53739 [==============================] - 188s 3ms/step - loss: 0.0993 1s - loss: 0.0 - ETA: 1s
Epoch 3/10
53739/53739 [==============================] - 187s 3ms/step - loss: 0.0858
Epoch 4/10
53739/53739 [==============================] - 187s 3ms/step - loss: 0.0792
Epoch 5/10
53739/53739 [==============================] - 187s 3ms/step - loss: 0.0733
Epoch 6/10
53739/53739 [==============================] - 186s 3ms/step - loss: 0.0669
Epoch 7/10
53739/53739 [==============================] - 182s 3ms/step - loss: 0.0599
Epoch 8/10
53739/53739 [==============================] - 184s 3ms/step - loss: 0.0527
Epoch 9/10
53739/53739 [==============================] - 191s 4ms/step - loss: 0.0458
Epoch 10/10
53739/53739 [==============================] - 196s 4ms/step - loss: 0.0399


In [105]:
dev = Corpus('../data/dev.json/*.json', 1000)

In [106]:
dev_x, dev_y = zip(*dev.xy())

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

In [107]:
dev_x = pad_sequences(dev_x, 50, padding='post', dtype=float)

In [108]:
dev_y = list(dev_y)

In [110]:
r2_score(dev_y, model.predict(dev_x))

0.26221508805730676

In [111]:
correct = Counter()
total = Counter()

for ab in dev.abstracts():
    
    x, _ = zip(*ab.xy())
    x = pad_sequences(x, 50, padding='post', dtype=float)
    
    preds = model.predict(x)
    order = list(preds[:,0].argsort().argsort())

    if sorted(order) == order:
        correct[len(order)] += 1

    total[len(order)] += 1

HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

In [112]:
for slen in sorted(correct.keys()):
    print(slen, correct[slen] / total[slen])

2 0.8247422680412371
3 0.40397350993377484
4 0.2265193370165746
5 0.07344632768361582
6 0.02564102564102564
8 0.016666666666666666


In [113]:
sum(correct.values()) / sum(total.values())

0.199